#### ✅ ขั้นตอนที่ Data Engineer ทำจริง


##### 1. ทำความเข้าใจ Requirement

- ถามว่า

  - ข้อมูลอยู่ไหน (API / DB / file / external bucket)?
  
  - Format อะไร (CSV, JSON, Parquet)?

  - Volume แค่ไหน (MB/GB)?

  - ต้องทำ Batch หรือ Streaming?

  - ความถี่ (daily, hourly)?

  - Load ไปไหน? (BigQuery, PostgreSQL, Data Lake)

📌 "เข้าใจโจทย์คือหัวใจหลักของ DE"

##### 2. เริ่มจาก Local Dev (เสมอ)

- ตั้งค่าบน local dev environment

  - สร้าง folder struturce ตามที่แนะนำ
  
  - สร้างไฟล์ notebook หรือ scripts สำหรับ ETL logic

  - โหลด sample data มาบางส่วนมาทำงานก่อน

- ทำ Clean/Transform ให้ตรงกับ business logic

  - เช่น Convert date format, drop nulls, filter valid row

✅ ETL จะเริ่มที่ local เพื่อให้ logic ถูกต้องก่อน แล้วจึง scale ไป cluster หรือ production

##### 🔁 สรุป Workflow ที่ใช้กันจริง (การทำบน Local)

🔹 Step 1: เริ่มจาก Notebook (EDA + Transform)
- ใช้ `eda_transform.ipynb` เพื่อ:

  - ดูโครงสร้าง raw data

  - ลอง clean, join, enrich, handle null, cast type ฯลฯ

  - ลอง logic transform ที่คิดไว้

  - แสดงตัวอย่างผลลัพธ์ เช่น `.show(5)`, `.summary()` หรือ `.write` ไปยัง temp folder

  🎯 จุดมุ่งหมาย: พิสูจน์ว่า logic ที่คิดไว้ "เวิร์ก" กับ data จริง

🔹 Step 2: ถ้า logic ใช้ได้ → ค่อยแยกเป็น Module

- ย้าย logic ที่ลองแล้ว ออกจาก Notebook

- แยกใส่ใน `etl/transform.py`, `etl/extract.py`, `etl/load.py`

- ทำให้ reusable, ทดสอบง่าย และ maintain ได้

- รันผ่าน `jobs/run_etl.py` โดยใช้ `SparkSession` และรับ config จาก `.yaml`

In [ ]:
# jobs/run_etl.py
from etl.extract import extract_csv # type: ignore
from etl.transform import clean_data # type: ignore
from etl.load import save_to_parquet # type: ignore

df = extract_csv(spark, config["input_path"]) # type: ignore
df_clean = clean_data(df)
save_to_parquet(df_clean, config["output_path"]) # type: ignore

##### 🎯 ทำไมใช้ Notebook ก่อนถึงดีที่สุด

| ข้อดี                                    | เหตุผล                                    |
| ---------------------------------------- | ----------------------------------------- |
| 🧪 เหมาะกับ trial-and-error              | ใช้ `.show()` และดูผลลัพธ์แบบ interactive |
| 🔄 ปรับ logic ได้ไว                      | ไม่ต้องรันจาก CLI ทุกครั้ง                |
| 👨‍💻 เห็นชัดว่าเปลี่ยนอะไรกับ data      | ทำให้ reasoning ดีขึ้น                    |
| 🧼 คัด logic ที่มั่นใจก่อนค่อยแยก module | ลด code ซ้ำซ้อนใน pipeline                |

##### ✅ สรุปแนวทางที่แนะนำ

- สร้างโครงสร้าง folder/project scaffolding ไว้ก่อน

- เริ่มจาก eda_transform.ipynb เพื่อทดลอง logic

- เมื่อ logic OK → ค่อยแยกออกเป็น module (extract/transform/load)

- ใช้ script run_etl.py เพื่อรัน end-to-end (simulate job จริง)

- หากอยาก realistic มากขึ้น → สร้าง daily partition, logger, test input set เพิ่มได้

##### 3. เขียน Script หรือ Modular Code


- แยกโค้ด ETL เป็นโมดูล เช่น

In [ ]:
  ├── etl/
  │   ├── extract.py
  │   ├── transform.py
  │   └── load.py

- ใช้ PySpark (หรือ Pandas ถ้าเล็ก) เขียน logic ให้ reusable

##### 4. ทดสอบ Pipeline

- ทดสอบด้วย sample data

- เช็กผลลัพธ์ของการ transform

- เช็กว่า load ไปที่ target แล้วหรือยัง

##### 5. จัดการ Environment และ Parameter

- ใช้ .env, config.yaml, CLI args เพื่อความยืดหยุ่น

- เช่น ``` python etl_job.py --run_date=2025-07-29 ```

##### 6. เตรียม Deployment/Production

- ถ้ารันเป็น batch job → ใช้ Airflow / cron / cloud scheduler

- ถ้าต้องรันบน cluster → ส่งขึ้น Spark cluster / Dataproc / EMR

- ถ้ารันบน cloud → เตรียม environment ให้เหมือน local แล้ว de

##### 📌 สรุป Flow ที่ใช้ฝึกได้ (เหมือนงานจริง)

| Step | Task                                           |
| ---- | ---------------------------------------------- |
| 1.   | ศึกษา source (API/DB/file)                     |
| 2.   | โหลด sample data มา local                      |
| 3.   | เขียน ETL logic (extract → transform → load)   |
| 4.   | Modularize code (scripts / jobs)               |
| 5.   | ทำ test run และ debug                          |
| 6.   | เตรียม production (parameter, config, logging) |
| 7.   | Deploy ไป cloud หรือ simulate batch run        |


#### เพิ่มเติม

##### วิธีแนวทางการ save ไฟล์ใน production 

✅ 1. ใช้ Parquet เป็นหลัก

เพราะ:

- ประหยัดพื้นที่ (columnar + compression)

- อ่านเร็วกว่า CSV มาก

- รองรับ schema

- ทำงานกับ Spark/BigQuery/Snowflake ได้ดี

In [ ]:
df.write.mode("overwrite").parquet("path/to/processed/clean_data")

✅ 2. ใช้ CSV เฉพาะเพื่อ export ไปให้คนอ่าน (Data Analyst / Excel / BI Tool)

เพราะ:

- ประหยัดพื้นที่ (columnar + compression)

- อ่านเร็วกว่า CSV มาก

- รองรับ schema

- ทำงานกับ Spark/BigQuery/Snowflake ได้ดี

In [ ]:
df.write.mode("overwrite").parquet("path/to/processed/clean_data")

✅ 3. แบ่งไฟล์แบบ partition (ถ้าข้อมูลใหญ่)

เพื่อให้:

- โหลดเร็ว (อ่านเฉพาะ partition ที่ต้องใช้)

- รองรับ query แบบกรองวัน/เดือน/year ได้ดี

In [ ]:
df.write.mode("overwrite").partitionBy("year", "month").parquet("path/to/partitioned")

✅ 4. เปิด compression (เช่น Snappy) เพื่อประหยัดพื้นที่

In [ ]:
df.write.mode("overwrite") \
  .option("compression", "snappy") \
  .parquet("path/to/compressed")

✅ 5. ตั้งชื่อ directory ตาม step / layer ใน pipeline

In [ ]:
data/
├── raw/             ← ข้อมูลดิบจาก API, DB
├── staged/          ← หลังจาก ingestion
├── processed/       ← หลัง transform / clean แล้ว
├── export_csv/      ← สำหรับ Analyst / external tools

✅ 6. ไม่เขียนไฟล์หลาย format ซ้อนกัน

- ในแต่ละขั้นจะเลือก format เดียว เพื่อความชัดเจน

- ถ้าต้องการหลาย format → แยกออกเป็นขั้นตอน export ต่างหาก

✅ 7. ระวัง overwrite กับ append

- `mode("overwrite")` ใช้สำหรับ dev/test หรือ batch ที่ต้อง replace

- `mode("append")` ใช้ใน production ที่เพิ่มข้อมูลทีละรอบ (เช่น real-time, daily batch)

##### สรุป Best Practices โดยย่อ

| แนวทาง                 | ใช้เมื่อ                          |
| ---------------------- | --------------------------------- |
| Parquet + Snappy       | Main pipeline, fast I/O           |
| CSV + header           | Export to business users          |
| Partition by date/user | Large dataset for efficient query |
| Overwrite vs Append    | ขึ้นกับ batch/realtime            |
| Clear folder structure | raw → staged → processed → output |